In [1]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

train_folder = 'DATASET/train'
test_folder = 'DATASET/test'


train_images = []
train_labels = []
train_labels1 = []

#contar el numero de archivos del directorio y si no hay archivos le asigno en la lista

for label in os.listdir(train_folder):
    label_folder = os.path.join(train_folder, label)
    list = os.listdir(label_folder) # dir is your directory path
    number_files = len(list)
    if (number_files) > 0:
        for filename in os.listdir(label_folder):
            img_path = os.path.join(label_folder, filename)
            img = cv2.imread(img_path)
            train_images.append(img)
            train_labels.append(label)
            train_labels1.append(label)


# Converción
train_images = np.array(train_images)
train_labels = np.array(train_labels)

# Convertir etiquetas a números
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)

# Segregación de entrenamiento
train_images, val_images, train_labels_encoded, val_labels_encoded = train_test_split(train_images, train_labels_encoded, test_size=0.2, random_state=42)

# Pixeles entre 1 y 0
train_images = train_images.astype('float32') / 255.0
val_images = val_images.astype('float32') / 255.0

# Definir la arquitectura de la CNN

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=train_images.shape[1:]))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# ejecutar modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrenaminento del modelo
model.fit(train_images, train_labels_encoded, epochs=10, batch_size=32, validation_data=(val_images, val_labels_encoded))


test_images = []
test_labels = []
test_labels1 = []

#guardo train_labels1 todos las clases de entrenamiento para en el test solo considerar las clase de entrenamiento


for label in os.listdir(test_folder):
    label_folder = os.path.join(test_folder, label)
    if (label in train_labels1):    
        for filename in os.listdir(label_folder):
            img_path = os.path.join(label_folder, filename)
            img = cv2.imread(img_path)
            test_images.append(img)
            test_labels.append(label)


# Convertir a matrices NumPy
test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Pixeles entre 1 y 0
test_images = test_images.astype('float32') / 255.0

# Codificar las etiquetas de prueba como números
test_labels_encoded = label_encoder.transform(test_labels)

# Realizar las predicciones en el conjunto de prueba
predictions = model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)

# Calculo e impresión de la matriz de confusión del modelo
confusion = confusion_matrix(test_labels_encoded, predicted_labels)
print('Matriz de confusión del modelo:')
print(confusion)

# Calculo y print de matriz de confusión del error en entrenamiento
train_predictions = model.predict(train_images)
train_predicted_labels = np.argmax(train_predictions, axis=1)
train_confusion = confusion_matrix(train_labels_encoded, train_predicted_labels)
print('Matriz de confusión del error en entrenamiento:')
print(train_confusion)

# Clculo e impresion eeror en prueba
test_predictions = model.predict(test_images)
test_predicted_labels = np.argmax(test_predictions, axis=1)
test_confusion = confusion_matrix(test_labels_encoded, test_predicted_labels)
print('Matriz de confusión del error en prueba:')
print(test_confusion)


Epoch 1/10
41/41 [==============================] - 77s 2s/step - loss: 1.5958 - accuracy: 0.5976 - val_loss: 0.9392 - val_accuracy: 0.6789
Epoch 2/10
41/41 [==============================] - 88s 2s/step - loss: 0.7600 - accuracy: 0.7299 - val_loss: 0.7082 - val_accuracy: 0.7187
Epoch 3/10
41/41 [==============================] - 88s 2s/step - loss: 0.3605 - accuracy: 0.8837 - val_loss: 0.4062 - val_accuracy: 0.8440
Epoch 4/10
41/41 [==============================] - 84s 2s/step - loss: 0.1518 - accuracy: 0.9526 - val_loss: 0.3796 - val_accuracy: 0.8440
Epoch 5/10
41/41 [==============================] - 87s 2s/step - loss: 0.1171 - accuracy: 0.9617 - val_loss: 0.3039 - val_accuracy: 0.9021
Epoch 6/10
41/41 [==============================] - 86s 2s/step - loss: 0.0646 - accuracy: 0.9770 - val_loss: 0.4633 - val_accuracy: 0.8563
Epoch 7/10
41/41 [==============================] - 86s 2s/step - loss: 0.0276 - accuracy: 0.9931 - val_loss: 0.3601 - val_accuracy: 0.8685
Epoch 8/10
41/41 [==